In [1]:
import os
from collections import Counter
import re

import glob

import pandas as pd
import numpy as np

import tweepy

from tqdm import tqdm

import spacy
from spacy.pipeline import Sentencizer
spacy.require_gpu()

nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words

## Lecture Data

### Preprocessing

In [2]:
tid2tm2begin2text = {}
tid2begin2text = {}
fr = open('data/' + 'allcaptions-2021-09-24.csv','r', encoding='utf8')
for line in tqdm(fr):
    arr = line.strip('\r\n').split('\t')
    if not len(arr) == 6: continue
    tid,tm,begin,text = arr[0],arr[1],arr[3],arr[5]
    if not tid in tid2tm2begin2text:
        tid2tm2begin2text[tid] = {}
    if not tm in tid2tm2begin2text[tid]:
        tid2tm2begin2text[tid][tm] = {}
    if not begin in tid2tm2begin2text[tid][tm]:
        tid2tm2begin2text[tid][tm][begin] = []
    tid2tm2begin2text[tid][tm][begin].append(text)
    if not tid in tid2begin2text:
        tid2begin2text[tid] = {}
    tid2begin2text[tid][begin] = text
fr.close()
fw = open('corpus_original.txt', 'w', encoding='utf8')
for tid,tm2begin2text in sorted(tid2tm2begin2text.items(),key=lambda x:x[0]):
    s = ' '
    for tm,begin2text in sorted(tm2begin2text.items(),key=lambda x:x[0]):
        for begin,texts in sorted(begin2text.items(),key=lambda x:x[0]):
            for text in texts:
                s += ' '+text
    s += ' '
    s = s.replace('\t',' ').replace('\r',' ')
    while '  ' in s:
        s = s.replace('  ',' ')
    fw.write(tid+'\t'+s[1:-1]+'\n')
fw.close()
fw = open('corpus_corrected.txt','w', encoding='utf8')
for tid,begin2text in sorted(tid2begin2text.items(),key=lambda x:x[0]):
    s = ' '    
    for begin,text in sorted(begin2text.items(),key=lambda x:x[0]):
        s += ' '+text
    s += ' '
    s = s.replace('\t',' ').replace('\r',' ')
    while '  ' in s:
        s = s.replace('  ',' ')
    fw.write(tid+'\t'+s[1:-1]+'\n')
fw.close()

4747258it [00:13, 345667.20it/s]


KeyboardInterrupt: 

In [ ]:
# nlp = spacy.load('en_core_web_sm')

# sentencizer = Sentencizer()
# sentencizer = nlp.add_pipe("sentencizer")

fw = open('nounchunks.txt','w')
fr = open('corpus_corrected.txt','r')
for line in tqdm(fr):
    arr = line.strip('\r\n').split('\t')
    tid = arr[0]
    text = arr[1]
    doc = nlp(text)
    n = len(list(doc.sents))
    fw.write(tid+'\t'+str(n)+'\n')
    sid = 0
    for text in doc.sents:
        _text_ = str(text)
        sid += 1
        fw.write('#s'+str(sid)+'\t'+_text_+'\n')        
        sent = nlp(_text_)
        s = ''
        for chunk in sent.noun_chunks:
            s += ';'+chunk.text
        if not s == '':
            fw.write('#c'+str(sid)+'\t'+s[1:]+'\n')
fr.close()
fw.close()

ValueError: [E003] Not a valid pipeline component. Expected callable, but got 'sentencizer' (name: 'None').[E004] If you meant to add a built-in component, use `create_pipe`: `nlp.add_pipe(nlp.create_pipe('sentencizer'))`

In [59]:
word2count = {}
word2df = {}
fr = open('corpus_corrected.txt','r')
for line in tqdm(fr):
    df_added = {}
    arr = line.strip('\r\n').split('\t')
    text = ' '+arr[1]+' '
    text = text.replace('.',' . ').replace(',',' , ').replace(';',' ; ').replace("'"," '")
    text = text.replace('?',' ? ').replace('!',' ! ').replace('(',' ( ').replace(')',' ) ')
    while '  ' in text:
        text = text.replace('  ',' ')
    text = text[1:-1].lower()
    for word in text.split(' '):
        if not word in word2count:
            word2count[word] = 0
        word2count[word] += 1
        if not word in word2df:
            word2df[word] = 0
        if not word in df_added:
            df_added[word] = True
            word2df[word] += 1
fr.close()
fw = open('vocabulary.txt','w')
for word,count in sorted(word2count.items(),key=lambda x:-x[1]):
    fw.write(word+'\t'+str(count)+'\t'+str(word2df[word])+'\n')
fw.close()

6713it [00:13, 500.34it/s]


In [60]:
words,counts,dfs = [],[],[]
wordsStop,countsStop = [],[]
fw = open('vocabulary-nonstop.txt','w')
fr = open('vocabulary.txt','r')
for line in tqdm(fr):
    arr = line.strip('\r\n').split('\t')
    word,count,df = arr[0],int(arr[1]),int(arr[2])
    words.append(word)
    counts.append(count)
    dfs.append(df)
    if word in stopwords or len(word) < 2:
        wordsStop.append(word)
        countsStop.append(count)
    else:
        fw.write(word+'\t'+str(count)+'\t'+str(df)+'\n')
fr.close()
fw.close()
n = len(words)
m = sum(counts)
nStop = len(wordsStop)
mStop = sum(countsStop)
print(n,nStop,100.*nStop/n)
print(m,mStop,100.*mStop/m)

67467it [00:00, 461989.11it/s]

67467 361 0.5350764077252583
35534319 23217282 65.33763036235477


In [8]:
nums = []
fr = open('nounchunks.txt','r')
for line in tqdm(fr):
    if not line.startswith('#'):
        arr = line.strip('\r\n').split('\t')
        nums.append(int(arr[1]))
fr.close()
fw = open('numsentences.txt','w')
for num in sorted(nums,key=lambda x:-x):
    fw.write(str(num)+'\n')
fw.close()

2910it [00:00, 2909517.20it/s]


In [25]:
nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words
chunk2count = {}
fr = open('nounchunks.txt','r')
for line in fr:
    if line.startswith('#c'):
        arr = line.strip('\r\n').split('\t')
        chunks = arr[1]
        for chunk in chunks.split(';'):
            words = chunk.split(' ')
            if len(words) > 0:
                while words[0].lower() in stopwords:
                    words.pop(0)
                    if len(words) == 0: break
            if len(words) > 0:
                while words[-1].lower() in stopwords:
                    words.pop(-1)
                    if len(words) == 0: break
            if len(words) > 0:
                _chunk_ = ''
                for word in words:
                    _chunk_ += ' '+word
                _chunk_ = _chunk_[1:]
                if not _chunk_ in chunk2count:
                    chunk2count[_chunk_] = 0
                chunk2count[_chunk_] += 1
fr.close()
fw1 = open('word2count.txt','w')
fw2 = open('phrase2count.txt','w')
for chunk,count in sorted(chunk2count.items(),key=lambda x:-x[1]):
    if ' ' in chunk:
        fw2.write(chunk+'\t'+str(count)+'\n')
    else:
        fw1.write(chunk+'\t'+str(count)+'\n')        
fw2.close()
fw1.close()

In [19]:
df_tid2course

,transcriptionid,videoid,Domain,termname,videoname,deptname,deptacronym,CourseNumber,CourseName,SectionName
0,000f5db7-9bc3-46f5-95a6-d694585ab399,c5c5bcfd-30c0-4a3c-a1d1-6e60ab5e030a,gvsu.edu,Spring 2021,Lecture 11: grep-wildcards,School of Computing and Information Systems,CIS,241,System-level Programming and Utilities,ALL
1,00168d74-e9cd-4ed9-9fd6-2945a6e8fd0a,d0ad8c6c-5f03-4d46-a282-598206e6f248,gvsu.edu,Spring 2021,Congestion Control -- Intro,School of Computing and Information Systems,CIS,457,Data Communications,ALL
2,001b0551-3d52-4eeb-b454-b1f87716d543,ffc7786a-1b78-4459-b16c-14adf08844e8,illinois.edu,Spring 2021,"CS 225 Lecture 2021-02-24 February 24, 2021",Computer Science,CS,225,Data Structures,AL1
3,001dba23-6b52-425d-be49-81364658488f,2832a7d8-e0f0-4f75-bf6b-fbb46df32df4,illinois.edu,Summer 2020,"CS 225-AL1 February 26, 2020",Computer Science,CS,225,Data Structures,AL1/AL2
4,002169f0-bfed-4209-86a5-312369a317bd,12250bd6-3129-4808-aac6-f8b6cb1189a1,illinois.edu,Fall 2020,UIUC CS 374 FA 20: 22.1.2. A quick pre-review ...,Computer Science,CS,374,Introduction to Algorithms & Models of Computa...,ANY
...,...,...,...,...,...,...,...,...,...,...
6880,ffc619b6-fc6e-4055-b90b-b5dc6aafb184,b00c6535-d6a3-4175-be32-8b7123ae24b8,illinois.edu,Spring 2019,CS 125 Spring 2019: Wed 2.13.2019. Constructor...,Computer Science,CS,125,Intro to Computer Science,AL2
6881,ffca023e-8d31-43e9-9757-d0959d3cc93f,c9e4fcb5-4994-4e40-b043-2f60cf03a096,illinois.edu,Fall 2021,"CS361: The Monty Hall Problem September 19, 2021",Computer Science,CS,361,Prob & Stat for Computer Science,ALL
6882,ffcfad32-b0a4-43a3-8111-f4d715d3c3e6,492cbf70-afe9-40b5-a4bb-6c4ed50c36cd,illinois.edu,Fall 2020,"CS374-ADC: Discussion 4A, Wednesday, September...",Computer Science,CS,374,(Live)Introduction to Algorithms & Models of C...,Live
6883,ffe3f31d-34b1-4889-a6de-58e2ca5e25e3,d2364571-4f39-4543-8ac1-5a3e99c3388b,illinois.edu,Fall 2020,UIUC CS 374 FA 20: 21.6.3. 2SAT,Computer Science,CS,374,Introduction to Algorithms & Models of Computa...,ANY


In [3]:
# parse tid to course name
df_tid2course = pd.read_csv("data/" + "transcriptions2courses-2021-09-24.csv", sep='\t')
# df_tid2course

tid2cn = dict(zip(df_tid2course['transcriptionid'], df_tid2course['deptacronym'] + "_" + df_tid2course['CourseNumber']))
cn2tid = {tid2cn[k]: k for k in tid2cn}
tid2cn

{'000f5db7-9bc3-46f5-95a6-d694585ab399': 'CIS_241',
 '00168d74-e9cd-4ed9-9fd6-2945a6e8fd0a': 'CIS_457',
 '001b0551-3d52-4eeb-b454-b1f87716d543': 'CS_225',
 '001dba23-6b52-425d-be49-81364658488f': 'CS_225',
 '002169f0-bfed-4209-86a5-312369a317bd': 'CS_374',
 '00284a96-1a55-4882-b283-cdd8daae345b': 'ECE_491',
 '003195b2-f1ec-4607-961e-0478183a0409': 'CS_361',
 '0040720f-3886-4dc4-a0ab-148bb31da28c': 'CS_233',
 '0041c995-61b1-4102-8bc9-773c333eb7bc': 'STAT_430',
 '004445f3-16da-42c1-8350-e2438c3c769d': 'CS_101',
 '005ba50a-666f-4da2-97e3-6f09a813931e': 'CS_357',
 '005ec5c0-e770-4566-8b8d-735de318ab49': 'ECE_408',
 '0065450c-daf5-45f8-a904-972f57ff8b8f': 'CS_173',
 '0067a78d-7c4e-45a9-808d-b86e05783279': 'CS_357',
 '006bb8f1-d722-4ef9-ad99-656f60bac7d4': 'CS_101',
 '007a4c69-a949-4550-b450-ecb4cd7a8b1e': 'CS_241',
 '00866d60-3f3b-47db-8f36-935a04b18a4e': 'IE_300',
 '008e9c45-fb96-4932-a932-c609d269c833': 'CS_361',
 '0098483f-bb8d-4828-b216-483e038cda7a': 'CS_357',
 '009c9eb3-b71a-4571-8437

In [7]:
# partition transcripts by course name
word2count = {}
fr = open('corpus_corrected.txt','r', encoding='utf8')
cn2texts = {}

for line in tqdm(fr):
    arr = line.strip('\r\n').split('\t')
    tid = arr[0]
    text = arr[1]
    if tid in tid2cn:
        cn = tid2cn[tid]

        if cn not in cn2texts:
            cn2texts[cn] = {}
        cn2texts[cn][tid] = text
fr.close()

for cn in tqdm(cn2texts):
    fw = open('data/' + 'course_captions_parsed/' + cn + '.txt','w', encoding='utf8')
    for tid in cn2texts[cn]:
        text = cn2texts[cn][tid]
        texts = []
        # get spacy postags
        doc = nlp(text)
        idx = 0
        for token in doc:
            texts.append("::".join([str(idx), token.text, token.pos_, token.ent_type_]))
            idx+=1

        fw.write(tid+'\t'+str(' '.join(texts))+'\n')
    fw.close()


6713it [00:00, 19514.50it/s]
 40%|███▉      | 27/68 [16:41<25:20, 37.09s/it]  


KeyboardInterrupt: 

### concept mining

In [8]:

# discover noun multi-grams
def _read_vocab():
    fr = open("vocab.txt", 'r', encoding='utf-8')
    word2freq = {}
    word2df = {}
    for line in fr:
        arr = line.strip().split('\t')
        word2freq[arr[0]] = int(arr[1])
        word2df[arr[0]] = int(arr[2])
    fr.close()
    return word2freq, word2df

def discover_noun_multi_grams(tokens, word2freq):
    # tokens is a list of parsed token
    rlt = []
    i = 0
    tmp = ""
    last = ""
    while i < len(tokens):
        _token = tokens[i].split('::')
        # print(_token)
        # if len(_token) != 9:
        #     i += 1
        #     continue
        if _token[1] not in word2freq or word2freq[_token[1]] < 3:
            i += 1
            continue
        if _token[2] in ["PROPN","NOUN"] and _token[1] != last: # avoid "vector vector"
            tmp += ' ' + _token[1]
            last = _token[1]
        else:
            if tmp.count(' ') > 1: # only need multi-grams
                multigram = tmp.strip().lower()
                rlt.append(multigram)
            tmp = ""
            last = ""
        i += 1
    return rlt

def _load_wiki_set():
    fr = open("week07/wiki_all.txt", 'r', encoding='utf-8')
    terms = [line.strip() for line in fr]
    fr.close()
    return set(terms)

In [11]:
def discover_concepts(cn):
    fr = open("data/course_captions_parsed/"+cn+".txt", 'r', encoding='utf-8')
    all_noun_multi_grams = set()
    multigram2freq = Counter()
    all_lectures = []
    # wiki_set = _load_wiki_set()
    word2freq, word2df =  _read_vocab()
    trancripid2lectid = {}
    # write lecture files
    for line in fr:
        arr = line.strip().split('\t')
        trancripid2lectid[arr[0]] = f"{cn}-{len(trancripid2lectid)}"
        all_lectures.append(trancripid2lectid[arr[0]])
        tokens = arr[1].split(' ')
        noun_multi_grams = discover_noun_multi_grams(tokens, word2freq)
        multigram2freq.update(noun_multi_grams)
        noun_multi_grams_set = set(noun_multi_grams)
        # all_noun_multi_grams |= noun_multi_grams_set
        tgt_dir = "week07/course_concepts/"+cn+'/'
        if not os.path.exists(tgt_dir):
            os.makedirs(tgt_dir)
        fw = open(tgt_dir+trancripid2lectid[arr[0]]+".md", 'w', encoding='utf-8')
        fw.write(f"#{cn}\n")
        fw.write("#lecture\n")
        for noun in noun_multi_grams_set:
            fw.write("[[" + noun + "]]\n") # create a link
        fw.close()
    fr.close()
    fw = open("week07/course_concepts/"+cn+"/statistics.txt",'w',encoding='utf-8')
    for k,v in multigram2freq.most_common():
        fw.write(f"{k}\t{v}\n")
    fw.close()
    # write concept files
    for noun, freq in multigram2freq.items():
        if '/' not in noun and freq > 2: # frequency filtering
        # if noun in wiki_set: # wiki filtering
            concept_fn = "week07/course_concepts/concepts/"+noun+".md"
            if os.path.exists(concept_fn):
                continue
            fw = open(concept_fn, 'w', encoding='utf-8')
            fw.write("#concept\n")
            fw.close()
    '''
    '''
    # wirte course files
    fw = open("week07/course_concepts/"+cn+".md", 'w', encoding='utf-8')
    fw.write("#course\n")
    for lect in all_lectures:
        fw.write("[["+lect+"]]\n")
    for noun, freq in multigram2freq.items():
        if '/' not in noun and freq > 2: # frequency filtering
            fw.write("[["+noun+"]]\n")

    fw.close()

# discover_concepts("CS_410")
# discover_concepts("CS_447")
# discover_concepts("ECE_448")
# discover_concepts("CS_361")
# discover_concepts("STAT_432")
discover_concepts("CS_446")


# discover_concepts("CS_357")


## Syllabus Data

### Load Data

In [3]:
tid2tm2begin2text = {}
tid2begin2text = {}

tm, begin = 0, 0

path = './data/syllabus/Fall2021/txt/*/'
files = glob.glob(path + '*.txt')

for filepath in files:
    with open(filepath,'r', encoding='utf8') as fr:
        text = '.'.join(fr.readlines())
    tid = filepath.split('\\')[-1].split('.')[0]  
    tm+=1
    begin+=1
    if not tid in tid2tm2begin2text:
        tid2tm2begin2text[tid] = {}
    if not tm in tid2tm2begin2text[tid]:
        tid2tm2begin2text[tid][tm] = {}
    if not begin in tid2tm2begin2text[tid][tm]:
        tid2tm2begin2text[tid][tm][begin] = []
    tid2tm2begin2text[tid][tm][begin].append(text)
    if not tid in tid2begin2text:
        tid2begin2text[tid] = {}
    tid2begin2text[tid][begin] = text

    fw = open('corpus_original.txt', 'w', encoding='utf8')
    for tid,tm2begin2text in sorted(tid2tm2begin2text.items(),key=lambda x:x[0]):
        s = ' '
        for tm,begin2text in sorted(tm2begin2text.items(),key=lambda x:x[0]):
            for begin,texts in sorted(begin2text.items(),key=lambda x:x[0]):
                for text in texts:
                    s += ' '+text
        s += ' '
        s = s.replace('\t',' ').replace('\r',' ').replace('\n','.')
        # replace consecutive dots into one dot
        dot_pattern = re.compile(r'\.{1,} *\.*')
        s = dot_pattern.sub('.', s)

        while '  ' in s:
            s = s.replace('  ',' ')

        # s=s.lower()
        fw.write(tid+'\t'+s[1:-1]+'\n')
    fw.close()

fw = open('corpus_corrected.txt','w', encoding='utf8')
for tid,begin2text in sorted(tid2begin2text.items(),key=lambda x:x[0]):
    s = ' '    
    for begin,text in sorted(begin2text.items(),key=lambda x:x[0]):
        s += ' '+text
    s += ' '
    s = s.replace('\t',' ').replace('\r',' ').replace('\n','.')
    # replace consecutive dots into one dot
    dot_pattern = re.compile(r' *\.{1,} *\.*')
    s = dot_pattern.sub('. ', s)

    while '  ' in s:
        s = s.replace('  ',' ')
    fw.write(tid+'\t'+s[1:-1]+'\n')
fw.close()

In [4]:
word2count = {}
word2df = {}
fr = open('corpus_corrected.txt','r')
for line in tqdm(fr):
    df_added = {}
    arr = line.strip('\r\n').split('\t')
    text = ' '+arr[1]+' '
    text = text.replace('.',' . ').replace(',',' , ').replace(';',' ; ').replace("'"," '")
    text = text.replace('?',' ? ').replace('!',' ! ').replace('(',' ( ').replace(')',' ) ')
    while '  ' in text:
        text = text.replace('  ',' ')
    text = text[1:-1].lower()
    for word in text.split(' '):
        word = word.strip()
        if not word in word2count:
            word2count[word] = 0
        word2count[word] += 1
        if not word in word2df:
            word2df[word] = 0
        if not word in df_added:
            df_added[word] = True
            word2df[word] += 1
fr.close()
fw = open('vocabulary.txt','w')
for word,count in sorted(word2count.items(),key=lambda x:-x[1]):
    fw.write(word+'\t'+str(count)+'\t'+str(word2df[word])+'\n')
fw.close()

23it [00:00, 481.82it/s]


In [5]:
words,counts,dfs = [],[],[]
wordsStop,countsStop = [],[]
fw = open('vocabulary-nonstop.txt','w')
fr = open('vocabulary.txt','r')
for line in tqdm(fr):
    arr = line.strip('\r\n').split('\t')
    word,count,df = arr[0],int(arr[1]),int(arr[2])
    words.append(word)
    counts.append(count)
    dfs.append(df)
    if word in stopwords or len(word) < 2:
        wordsStop.append(word)
        countsStop.append(count)
    else:
        fw.write(word+'\t'+str(count)+'\t'+str(df)+'\n')
fr.close()
fw.close()
n = len(words)
m = sum(counts)
nStop = len(wordsStop)
mStop = sum(countsStop)
print(n,nStop,100.*nStop/n)
print(m,mStop,100.*mStop/m)

4654it [00:00, 465555.84it/s]

4654 272 5.844434894714224
91758 47431 51.69140565400292


In [6]:
# parse tid to course name
tid2cn = {}
for filepath in files:
    filename = filepath.split('\\')[-1].split('.')[0]  
    tid2cn[filename] = filename[:3]

tid2cn

{'ACC201-01 Syllaus_8-28-21': 'ACC',
 'ACC202-01 Syllaus_8-28-21': 'ACC',
 'ACC203-01 Syllaus_8-28-21': 'ACC',
 'ACC301-01 Syllaus_8-26-21': 'ACC',
 'ACC350-01 Syllaus_8-26-21': 'ACC',
 'BUS101-01 Syllabus_8-28-21': 'BUS',
 'BUS181-01 Syllabus_8-27-21': 'BUS',
 'BUS181-02 Syllabus_8-27-21': 'BUS',
 'BUS211-01 Syllabus_8-28-21': 'BUS',
 'BUS221-01 Syllabus_8-27-21': 'BUS',
 'BUS290-GSR220 Syllabus_8-30-21': 'BUS',
 'BUS322-01 Syllabus_8-28-21': 'BUS',
 'BUS341-01 Syllabus_8-28-21': 'BUS',
 'BUS361-01 Syllabus_8-26-21': 'BUS',
 'BUS371-01 Syllabus_8-26-21': 'BUS',
 'BUS381-01 Syllabus_8-26-21': 'BUS',
 'BUS390-01 Syllabus_8-26-21': 'BUS',
 'BUS491-01_Syllabus_8-26-21': 'BUS',
 'BUS492-01_Syllabus_8-29-21': 'BUS',
 'RMI201_01 Syllabus_8-27-21': 'RMI',
 'RMI201_02 Syllabus_8-27-21': 'RMI',
 'RMI305_01 Syllabus_8-27-21': 'RMI',
 'RMI310_01 Syllabus_8-27-21': 'RMI'}

In [7]:
# partition transcripts by course name
word2count = {}
fr = open('corpus_corrected.txt','r', encoding='utf8')
cn2texts = {}

for line in tqdm(fr):
    arr = line.strip('\r\n').split('\t')
    tid = arr[0]
    text = arr[1]
    if tid in tid2cn:
        cn = tid2cn[tid]

        if cn not in cn2texts:
            cn2texts[cn] = {}
        cn2texts[cn][tid] = text
fr.close()

for cn in tqdm(cn2texts):
    fw = open('data/' + 'syllabus_parsed/' + cn + '.txt','w', encoding='utf8')
    for tid in cn2texts[cn]:
        text = cn2texts[cn][tid]
        texts = []
        # get spacy postags
        doc = nlp(text)
        idx = 0
        for token in doc:
            texts.append("::".join([str(idx), token.text, token.pos_, token.ent_type_]))
            idx+=1

        fw.write(tid+'\t'+str(' '.join(texts))+'\n')
    fw.close()


23it [00:00, 11488.51it/s]
100%|██████████| 3/3 [00:08<00:00,  2.68s/it]


### concept mining

In [10]:

# discover noun multi-grams
def _read_vocab():
    fr = open("vocab.txt", 'r', encoding='utf-8')
    word2freq = {}
    word2df = {}
    for line in fr:
        arr = line.strip().split('\t')
        # print(arr)
        word2freq[arr[0]] = int(arr[1])
        word2df[arr[0]] = int(arr[2])
    fr.close()
    return word2freq, word2df

def discover_noun_multi_grams(tokens, word2freq):
    MAX_LEN = 5
    # tokens is a list of parsed token
    rlt = []
    i = 0
    tmp = ""
    last = ""
    while i < len(tokens):
        _token = tokens[i].split('::')
        # print(_token)
        # if len(_token) != 9:
        #     i += 1
        #     continue
        text = _token[1].lower()
        # if text == 'financial':
        #     print(_token)
        #     print(word2freq[text])

        if (text not in word2freq) or (word2freq[text] < 10):
            i += 1
            continue
        # Entity constraint
        if _token[3] in ['ORG', 'GPE', 'LAW']:
            tmp += ' ' + text
            last = text
        elif _token[2] in ["PROPN","NOUN"] and text != last: # avoid "vector vector"
            tmp += ' ' + text
            last = text
        else:
            if (tmp.count(' ') > 1) and (tmp.count(' ') < MAX_LEN): # only need multi-grams
                multigram = tmp.strip().lower()
                rlt.append(multigram)
            tmp = ""
            last = ""
        i += 1
    return rlt

def _load_wiki_set():
    fr = open("syllabus/wiki_all.txt", 'r', encoding='utf-8')
    terms = [line.strip() for line in fr]
    fr.close()
    return set(terms)

In [11]:
def discover_concepts(cn):
    fr = open("data/syllabus_parsed/"+cn+".txt", 'r', encoding='utf-8')
    all_noun_multi_grams = set()
    multigram2freq = Counter()
    all_lectures = []
    # wiki_set = _load_wiki_set()
    word2freq, word2df =  _read_vocab()
    trancripid2lectid = {}
    # write lecture files
    for line in fr:
        arr = line.strip().split('\t')
        trancripid2lectid[arr[0]] = f"{cn}-{len(trancripid2lectid)}"
        all_lectures.append(trancripid2lectid[arr[0]])
        tokens = arr[1].split(' ')
        noun_multi_grams = discover_noun_multi_grams(tokens, word2freq)
        multigram2freq.update(noun_multi_grams)
        noun_multi_grams_set = set(noun_multi_grams)
        # all_noun_multi_grams |= noun_multi_grams_set
        tgt_dir = "syllabus/course_concepts/"+cn+'/'
        if not os.path.exists(tgt_dir):
            os.makedirs(tgt_dir)
        fw = open(tgt_dir+trancripid2lectid[arr[0]]+".md", 'w', encoding='utf-8')
        fw.write(f"#{cn}\n")
        fw.write("#lecture\n")
        for noun in noun_multi_grams_set:
            fw.write("[[" + noun + "]]\n") # create a link
        fw.close()
    fr.close()
    fw = open("syllabus/course_concepts/"+cn+"/statistics.txt",'w',encoding='utf-8')
    for k,v in multigram2freq.most_common():
        fw.write(f"{k}\t{v}\n")
    fw.close()
    # write concept files
    for noun, freq in multigram2freq.items():
        if '/' not in noun and freq > 2: # frequency filtering
        # if noun in wiki_set: # wiki filtering
            concept_fn = "syllabus/course_concepts/concepts/"+noun+".md"
            if os.path.exists(concept_fn):
                continue
            fw = open(concept_fn, 'w', encoding='utf-8')
            fw.write("#concept\n")
            fw.close()
    '''
    '''
    # wirte course files
    fw = open("syllabus/course_concepts/"+cn+".md", 'w', encoding='utf-8')
    fw.write("#course\n")
    for lect in all_lectures:
        fw.write("[["+lect+"]]\n")
    for noun, freq in multigram2freq.items():
        if '/' not in noun and freq > 2: # frequency filtering
            fw.write("[["+noun+"]]\n")

    fw.close()


discover_concepts("ACC")
discover_concepts("BUS")
discover_concepts("RMI")


